In [4]:
import secrets

import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from openrouteservice import client
import shapely

import requests
import json
import time
import sqlite3
from datetime import date

In [6]:
#Global Vars
CENSUS_KEY = secrets.CENSUS_API_KEY
ORS_KEY = secrets.ORS_API_KEY

overpass_url = "http://overpass-api.de/api/interpreter?"
overpass_query_markets = '''[out:json]
[timeout:25]
;
area(3600175905)->.searchArea;
(
  node
    ["shop"="supermarket"]
    (area.searchArea);
  way
    ["shop"="supermarket"]
    (area.searchArea);
  relation
    ["shop"="supermarket"]
    (area.searchArea);
  node
    ["shop"="grocery"]
    (area.searchArea);
  way
    ["shop"="grocery"]
    (area.searchArea);
  relation
    ["shop"="grocery"]
    (area.searchArea);
  node
    ["shop"="greengrocer"]
    (area.searchArea);
  way
    ["shop"="greengrocer"]
    (area.searchArea);
  relation
    ["shop"="greengrocer"]
    (area.searchArea);
);
out center;
>;
out skel qt;
'''

AttributeError: module 'secrets' has no attribute 'CENSUS_API_KEY'

In [ ]:
def open_cache(cache_path=None):
    '''
        Opens the cache with the file path provided as a dictionary; if no cache is present,
        creates a cache dictionary.

        Returns the resultant cache dictionary.

        Parameters
        ----------
        cache_path: str
            The path to a cache file, if such a file exists.

        Returns
        -------
        dict
            A dictionary containing cached information stored in the form of a json.
        '''
    try:
         with open(cache_path, 'r') as cache_file:
                cache = json.load(cache_file_contents)
    except:
        cache = {}
    return cache


def save_cache(cache, cache_path):
    '''
        Saves a cache dictionary to the filepath provided.

        Parameters
        ----------
        cache: dict
            A dictionary containing cached webpage information.
        
        cache_path: str
            The file path where the cache is to be saved.


        Returns
        -------
        None
        '''
      with open(cache_path, 'w') as cache_file:
            json.dump(cache, cache_file, indent=2)




In [ ]:
def call_API_with_cache(url, params=None, cache=None):
    '''
        Manages API calls using the provided cache of API-derived data.

        Parameters
        ----------
        url: string
            The location of a resource to be requested.

        params: dict
            A dictionary containing search parameters; defaults to None.
            
        cache: dict
            A dictionary containing previously obtained data.
            

        Returns
        -------
        dict
            The JSON returned by the API call, formatted as a dictionary.
        '''
    if params is not None:
        key = construct_unique_key(params, url)
    else:
        key = url
        
    if cach is not None and key in cache.keys():
        print(f"Using Cache: {url}")
        content = cache[key]
        return content
    else:
        print(f"Fetching: {url}")
        if params is not None:
            content = requests.get(url=url, params=params).json()
        else:
            content = requests.get(url=url).json()
            
        cache[key] = content
        save_cache(cache)
            
    return content

def construct_unique_key(params, api_url=API_URL):
    '''
        Constructs a unique key for a webpage (to be used in this program's
        cache) from supplied parameters and an API URL.

        Parameters
        ----------
        params: dict
            A dictionary containing search parameters.

        api_url: string
            The location of an API. Defaults to global API_URL.


        Returns
        -------
        str
            A unique key.
        '''
    param_strings = []
    connector = '_'
    for k in params.keys():
        param_strings.append(f'{k}_{params[k]}')
    unique_key = api_url + connector + connector.join(param_strings)
    return unique_key

In [1]:
def get_markets(refresh=False):
    '''TODO: Docstring
    
    Fetches market data, saves and returns as GeoJSON'''
    
    geographic_elements = {'type':'FeatureCollection',
                      'name':'markets',
                      'features':[]}
    
    if refresh = True:
        market_cache = None
    else:
        market_cache = open_cache(MARKET_CACHE_PATH)
    
    results = call_API_with_cache(url=overpass_url, params={'data':overpass_query_markets}, cache=market_cache)
    
    for element in results['elements']:
    if 'tags' in element:
        geodict = {'type':'Point'}
        propdict = {'id':element['id']}

        if element['type'] == 'node' and 'tags' in element:
            lon = element['lon']
            lat = element['lat']
            geodict['coordinates'] = [lon, lat]
            
        elif 'center' in element:
            lon = element['center']['lon']
            lat = element['center']['lat']
            geodict['coordinates'] = [lon, lat]
    
        for key, value in element['tags'].items():
            propdict[key] = value
        
        geographic_elements['features'].append({'type':'Feature',
                                   'geometry':geodict,
                                   'properties':propdict})

    with open('./markets.geojson', 'w') as file:
        json.dump(geographic_elements, file, indent=2)
    
    return geographic_elements
    

SyntaxError: invalid syntax (<ipython-input-1-766621b39913>, line 2)

In [ ]:
def divide_features(feature_df, n, geometry_col, id_col):
    '''TODO: Write Docstring'''
    ids_with_locations = {}

    feature_df['geom_reformat'] = feature_df[geometry_col].apply(lambda location: [location.x, location.y])
    
    df_chunks = np.array_split(feature_df[[id_col, 'geom_reformat']], math.trunc(feature_df.shape[0]/5)+1)
    
    for chunk in df_chunks:
        id_string = ''
        
        for item in chunk[id_col].tolist():
            id_string += f'{item}_'
        
        location_list = chunk['geom_reformat'].tolist()
        
        ids_with_locations[id_string] = location_list
        

    return ids_with_locations



In [ ]:
def get_isochrones(points):
    points_data = gpd.GeoDataFrame(points)
    clnt = client.Client(key=ORS_KEY)
    
    segments = divide_features(markets, 5, 'geometry', 'id')

    params_iso = {'location_type':'destination',
              'range': [600, 420, 300], #420/60 = 7 mins
              'range_type': 'time',
              'attributes': ['area', 'reachfactor', 'total_pop'], # Get attributes for isochrones
              'smoothing': 5
             }

    head_iso = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': ORS_KEY,
        'Content-Type': 'application/json; charset=utf-8'
    }

    isochrones = {'type':'FeatureCollection',
                     'features':[]}
    
    index = []

    for id_string, locations in segments.items():
        params_iso['locations'] = locations
        params_iso['id'] = id_string
        
        id_list = np.repeat(id_string.split(sep='_'), len(params_iso['range']))
        index += id_list

        try:
            isos = requests.post(ORS_URL, json=params_iso, headers=head_iso).json()

            i = 0
            for feature in isos['features']:
                feature['properties']['id'] = id_list[i]
                i += 1
                isochrones['features'].append(feature)
                print(feature)

        except:
            time.sleep(61)

            isos = requests.post(ORS_URL, json=params_iso, headers=head_iso).json()

            i = 0
            for feature in isos['features']:
                feature['properties']['id'] = id_list[i]
                i += 1
                isochrones['features'].append(feature)

    return {'index': index, 'isochrones': isochrones}
    

In [ ]:
def market_isochrones_with_cache(market_geographies):
    #SET UP TO WORK WITH CACHE
    pass

In [ ]:
#FOR 507 FINAL
def open_cache(cache_name, cache_path):
    '''TODO: Docstring'''
    with open(cache_path) as cache:
        cache_name = json.load(cache)
    
    return cache_name

def save_cache(cache_name, cache_path):
    '''TODO: Docstring'''
    with open(cache_path, 'w') as file:
        json.dump(cache_name, file, indent=2)

def make_market_database(url=overpass_url, query=overpass_query_markets):
    '''TODO: Docstring'''
    
    results = requests.get(url=url, params={'data':query}).json()
    

In [2]:
if __name__ == "main":
    
    #Get markets
    markets = gpd.get_markets()
    
    

SyntaxError: EOL while scanning string literal (<ipython-input-2-64dc2eee03d0>, line 5)